In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
from feat import Detector
import glob
import os
import cv2
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
import pickle

In [ ]:
# Initialize the detector

detector = Detector(
    face_model="retinaface",
    landmark_model="mobilefacenet",
    au_model='xgb',
    emotion_model="resmasknet",
    facepose_model="img2pose",
    device="cpu",
)

In [ ]:
# Initialize the DataFrame to store AU data
aus_data = pd.DataFrame()

# Define the path to the dataset directory
dataset_path = "../Desktop/dataset"

# Loop through each emotion subfolder in the dataset directory
for emotion_folder in os.listdir(dataset_path):
    emotion_path = os.path.join(dataset_path, emotion_folder)

    # Check if it's a directory
    if os.path.isdir(emotion_path):
        # Process each image in the subfolder
        for filename in glob.glob(os.path.join(emotion_path, "*.png")):
            img = cv2.imread(filename)
            detection = detector.detect_image(filename)

            # Extract AU data
            aus = detection.aus
            au_data = pd.DataFrame(aus)

            # Add emotion label from folder name
            au_data['emotion'] = emotion_folder

            # Rearrange columns to have 'emotion' first
            cols = ['emotion'] + [col for col in au_data.columns if col != 'emotion']
            au_data = au_data[cols]

            # Append to the main DataFrame
            aus_data = pd.concat([aus_data, au_data], ignore_index=True)

# Save the AU data to a CSV file
aus_data.to_csv("aus.csv", index=False)

In [ ]:
# Load the dataset
data = pd.read_csv('aus.csv')

# Ensure that only numeric columns are included in the mean calculation
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

# Encoding the 'emotion' column
label_encoder = LabelEncoder()
data['emotion'] = label_encoder.fit_transform(data['emotion'])

# Feature Scaling for AUs
scaler = StandardScaler()
AU_columns = [col for col in data.columns if col.startswith('AU')]
data[AU_columns] = scaler.fit_transform(data[AU_columns])

# Split the dataset into train/validation/test sets (70/20/10 split)
labels = data['emotion']
features = data.drop('emotion', axis=1)

# Splitting the data
train_val_features, test_features, train_val_labels, test_labels = train_test_split(
    features, labels, test_size=0.1, random_state=42, stratify=labels)
train_features, val_features, train_labels, val_labels = train_test_split(
    train_val_features, train_val_labels, test_size=(0.2/0.9), random_state=42, stratify=train_val_labels)

# Grid search for SVM
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid_search.fit(train_features, train_labels)

# Best parameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Evaluate on test set
best_svm_model = grid_search.best_estimator_
test_accuracy = best_svm_model.score(test_features, test_labels)

print(f"Best Parameters: {best_params}")
print(f"Best Training Score: {best_score}")
print(f"Test Set Accuracy: {test_accuracy}")


In [ ]:


# Save the trained model to a file
with open('svm_model.pkl', 'wb') as file:
    pickle.dump(best_svm_model, file)

# Optionally, you can also save the scaler and label encoder if you need them for preprocessing in the future
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)


In [13]:
# Load the model from the file
with open('svm_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Load the scaler and label encoder if needed
with open('scaler.pkl', 'rb') as file:
    loaded_scaler = pickle.load(file)

with open('label_encoder.pkl', 'rb') as file:
    loaded_label_encoder = pickle.load(file)


In [15]:
# Example: Assuming 'new_au_data' is your extracted AU features from Py-FEAT
new_au_data_scaled = loaded_scaler.transform(new_au_data)  # Use the same scaler as during training
predictions = loaded_model.predict(new_au_data_scaled)
predicted_emotions = loaded_label_encoder.inverse_transform(predictions)

predicted_emotions[0]

'angry'